In [2]:
#%pip install googlemaps
import googlemaps
import time
import pandas as pd
import os
from dotenv import load_dotenv

In [28]:
load_dotenv()
gmaps = googlemaps.Client(key=os.getenv('GOOGLE_MAPS_API_KEY'))
query = "Winterthur, Switzerland"

In [20]:
def restaurant_response(response_items):
    restaurant_responses = []
    for item in response_items:
        item_dict = {}
        item_dict['name'], item_dict['place_id'], item_dict['price_level'], item_dict['rating'], item_dict['user_ratings_total'], item_dict['address'] = item.get('name'), item.get('place_id'), item.get('price_level'), item.get('rating'), item.get('user_ratings_total'), item.get('formatted_address')
        restaurant_responses.append(item_dict)
    return restaurant_responses

In [29]:
restaurants = []
query_result = gmaps.places(query, type="restaurant", radius=10_000)
restaurants.extend(restaurant_response(query_result['results']))
while query_result.get('next_page_token') is not None:
    print(f"Finding more restaurants in Winterthur, Switzerland")
    time.sleep(2)
    query_result = gmaps.places(page_token=query_result['next_page_token'])
    response_items = query_result['results']
    restaurants.extend(restaurant_response(response_items))

Finding more restaurants in Winterthur, Switzerland
Finding more restaurants in Winterthur, Switzerland


In [30]:
pd.DataFrame(restaurants).to_csv('restaurants.csv', index=False)

In [40]:
#%pip install tqdm
import requests
from tqdm import tqdm

Note: you may need to restart the kernel to use updated packages.


In [48]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Referer': 'http://www.google.com'
}

osm_join_data = {
    'google_places_id': [],
    'osm_id': [],
}

osm_restaurants = []

restaurant_iter = tqdm(restaurants)
amenity_types = ['restaurant', 'cafe', 'fast_food', 'bar', 'pub', 'food_court', 'ice_cream', 'biergarten', 'bbq', 'deli', 'bakery', 'confectionery', 'pastery', 'tea', 'wine', 'juice', 'coffee', 'food']

for restaurant in restaurant_iter:
    restaurant_iter.set_description(f"Finding OSM IDs")
    restaurant_iter.set_postfix({'restaurant': restaurant['name']})
    response = requests.get(f'https://nominatim.openstreetmap.org/search?q={restaurant["address"]}&format=json', headers=headers)
    if response.status_code == 200:
        for result in response.json():
            if result['class'] == 'amenity' and result['type'] in amenity_types:
                osm_join_data['google_places_id'].append(restaurant['place_id'])
                osm_join_data['osm_id'].append(result['place_id'])
                osm_restaurants.append(result)
                
    else:
        print(f"Error: {response.status_code}")
        print(response.content)
    time.sleep(2)

Finding OSM IDs: 100%|██████████| 60/60 [02:15<00:00,  2.26s/it, restaurant=Goldeneshorn]                                                   


In [49]:
pd.DataFrame(osm_join_data).to_csv('google_osm.csv', index=False)

In [50]:
pd.DataFrame(osm_restaurants).to_csv('osm_restaurants.csv', index=False)